# &ldquo;OFFO&rdquo; 3&times;3&times;3 supercell generation

This notebook generates 3&times;3&times;3 TiOF<sub>2</sub> supercells following the &ldquo;OFFO&rdquo;-type ordering proposed by Brink <i>et al.</i> for NbO<sub>2</sub>F [1] (and subsequently for TaO<sub>2</sub>F [2]).

The supercells are generated so that each column has either a OFFO, FOFF, or FFOF sequence; i.e., on average, the oxygen and fluorine positions are uncorrelated between pairs of $\left<001\right>$ strings, as per the model of Brink <i>et al.</i> [1].

[1] Brink <i>et al.</i> J. Sol. Stat. Chem. <b>166</b>, 73–80 (2002): [link](https://doi.org/10.1006/jssc.2002.9562)</br>
[2] Morelock <i>et al.</i> Chem. Mater. <b>25</b>, 1900–1904 (2013): [link](https://.doi.org/10.1021/cm400536n)

In [1]:
import numpy as np
from pymatgen.core import Structure, Lattice

In [2]:
# Generate a reference ReO3-type 3x3x3 supercell

lattice = Lattice.from_parameters(a=3.8076, b=3.8076, c=3.8076,
                                  alpha=90.0, beta=90.0, gamma=90.0)

s = Structure.from_spacegroup(sg='Pm-3m',
                              lattice=lattice,
                              species=('Ti', 'X'),
                              coords=((0.0, 0.0, 0.0),
                                      (0.0, 0.0, 0.5))) * (3,3,3)

In [3]:
def align_coords(c: np.array) -> np.array:
    """Rounds an array of floats to 3 decimal places,
    and returns the remainder if diving by 1.0
    
    Args:
        c (np.ndarray): The array to be "aligned".
        
    Returns:
        np.ndarray
        
    """
    return np.remainder(np.round(c, 3), 1)

In [4]:
def get_column_indices(s: Structure,
                       cation_species: str) -> np.array:
    """Returns a NxM numpy array of site indices, where each row gives the
    indices for the anions along one x, y, or z column within an input ReO3-type structure.
    
    
    Args:
        s (Structure): The input ReO3-type structure. Anions should be of species type "X0+".
        cation_species (str): The species string identifying the cations (e.g., "Ti").
        
    Returns:
        np.ndarray
        
    """
    cation_coords = align_coords(
        np.vstack(
            [site.frac_coords
             for site in s.sites
             if site.species_string==cation_species]
        )
    )
    all_column_indices = []
    axes=[0, 1, 2]
    for d in axes[-1::-1]:
        ad = np.array([a for a in axes if a!=d])
        c = np.unique(cation_coords[:,ad], axis=0)
        for cd in c:
            column_indices = []
            for i, site in enumerate(s.sites):
                if site.species_string=='X0+':
                    if np.all(align_coords(site.frac_coords[ad]) == cd):
                        column_indices.append(i)
            all_column_indices.append(column_indices)
    return np.array(all_column_indices)

In [5]:
def random_OFFO_columns_from_structure(s: Structure,
                                       cation_species: str = 'Ti',
                                       majority_anion_species: str = 'F',
                                       minority_anion_species: str = 'O'
                                      ) -> Structure:
    """Takes an input ReO3-type pymatgen Structure and changes the anions to species
    A and B, with only one anion in each x, y, and d column the minority anions species.
    
    Args:
        s (Structure):  The input ReO3-type structure. Anions should be of species type "X0+".
        cation_species (str): The species string identifying the cations (e.g., "Ti").
        majority_anion_species (optional[str]): Species string for the majority anion species. Default is "F".
        minority_anion_species (optional[str]): Species string for the minority anion species. Default is "O".
        
    Returns:
        Structure
        
    """
    s_copy = s.copy()
    x_indices = get_column_indices(s_copy, cation_species=cation_species)
    o_indices = np.array([np.random.choice(column) for column in x_indices])
    for site in s_copy.sites:
        if site.species_string == 'X0+':
            site.species = majority_anion_species
        for i in o_indices:
            s_copy[i].species = minority_anion_species
    s_copy.sort()
    return s_copy

In [6]:
for i in range(20):
    filename = f'OFFO_{i+1:02d}.poscar'
    OFFO_structure = random_OFFO_columns_from_structure(s)
    OFFO_structure.to(fmt='poscar', filename=filename)